In [5]:
import os
from typing import TypedDict
import time
from langgraph.graph import StateGraph, END
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from dotenv import load_dotenv

In [3]:
class State(TypedDict): 
    query: str
    is_math: bool
    result: str 

# ╔═══════════════════════════════════╗
# ║      Tool Selection Flow          ║
# ╚═══════════════════════════════════╝
#
#       [START]
#          │
#          ▼
#    ┌───────────┐
#    │ classify  │ Check if math query
#    └─────┬─────┘ Sets: is_math boolean
#          │
#     ┌────┴────┐
#     │ router()│ Routes based on
#     └────┬────┘ state["is_math"]
#          │
#    ┌─────┴─────┐
#    ▼           ▼
# ┌──────────┐ ┌─────────┐
# │calculator│ │ general │
# │ (is_math)│ │(!is_math)│
# │   🤖LLM  │ │ message │
# └────┬─────┘ └────┬────┘
#      ▼            ▼
#    [END]        [END]
#
# KEY CONCEPTS:
# - First LLM integration!
# - LLM acts as calculator tool
# - Router selects appropriate tool
# - Different paths for different queries

In [4]:
# Define calculator tool
@tool
def calculator_tool(expression: str) -> str:
    """
    Evaluates basic mathematical expresseions
    
    Args:
        expression: A mathmetical expression like "25+17"
    
    Returns:
        calculated result as a string
    """

    try:
        allowed_names = {
            'abs': abs, 'round': round, 'min': min, 'max': max,
            'sum': sum, 'len': len, 'int': int, 'float': float
        }
        result = eval(expression, {"__builtins__": {}}, allowed_names)
        return str(result)
    except Exception as e:
        return f"Error calculating: {str(e)}"
    

In [7]:
import keyword
load_dotenv()

llm = ChatOpenAI(
        model="gpt-4.1-mini",
        api_key=os.getenv("OPENAI_API_KEY"),
        base_url=os.getenv("OPENAI_API_BASE"),
        temperature=0   # Return deterministic result for math accuracy
    )
llm_with_calculator = llm.bind_tools([calculator_tool])

# Classify if query is mathmethical expression
def classfigy_node(state: State):
    query_lower = state['query'].lower()
    math_keywords = ["+", "-", "*", "/", "plus", "minus", "divided", "calculate", "sum"]
    is_math = any(keyword in query_lower for keyword in math_keywords)

    if is_math:
        print(" ✅ Detected mathmetical query")
    else:
        print(" ❕ Non-mathematical query")
    return {"is_math": is_math}

In [19]:
def router(state: State):
    """Route to calculator or general response"""
    if state["is_math"]:
        return "calculator"
    return "general"

def calculator_node(state: State):
    """Uses LLM with calculator tool"""
    print("Invoking calculator tool\n")
    time.sleep(2)
    prompt = f"Calculate the following using the calculator tool: {state['query']}" 
    response = llm_with_calculator.invoke(prompt)

    if hasattr(response, 'tool_calls') and response.tool_calls:
        tool_call = response.tool_calls[0]
        print(f"Tool called: {tool_call['name']}")
        print(f"Expression: {tool_call['args'].get('expressions', '')}")

        result = calculator_tool.invoke(tool_call['args'])
        answer = result 
    else:
        answer = response.content.strip()

    print("Calculator returned result\n")
    time.sleep(1)
    
    return {"result": f"Answer: {answer}"}

def general_response_node(state: State):
    time.sleep(2)
    return {"result": "This is not a math question. Plase ask a calculation"}

print("Building calculator graph\n")

workflow = StateGraph(State)

workflow.add_node("classify", classfigy_node)
workflow.add_node("calculator", calculator_node)
workflow.add_node("general", general_response_node)

workflow.set_entry_point("classify")
workflow.add_conditional_edges(
    "classify",
    router,
    {
        "calculator": "calculator",
        "general": "general"
    }
)

workflow.add_edge("calculator", END)
workflow.add_edge("general", END)

app = workflow.compile()

Building calculator graph



In [20]:
test1 = app.invoke({
    "query": "What is 3+10*2-4",
    "is_math": False,
    "result": ""
})
print(f"Result: {test1['result']}")

 ✅ Detected mathmetical query
Invoking calculator tool

Tool called: calculator_tool
Expression: 
Calculator returned result

Result: Answer: 19


In [23]:
test2 = app.invoke({
    "query": "What is the weather today?",
    "is_math": False,
    "result": ""
})

print(f"Result:{test2['result']}")

 ❕ Non-mathematical query
Result:This is not a math question. Plase ask a calculation
